# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [24]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [26]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [5]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [6]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [33]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [37]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

For the prompts as a literature expert, I experiemented with zero-shot inference and few-shots inference to see the difference. In both cases the bot could give me the answer to my question, however, with zero-shot, the answer came in the form of a long drawn out answer that provides a lot of extraneous information about the book that I didn't ask for, whereas with few-shots, the answer was correct, and in the format of the inference references provided. So it clearly learned which information was specifically being asked for, which demonstrates that is learned by example of how to answer a question that is asked this way. When prompted with few-shot examples of literature, but asked about pizza instead, it behaves like a bot would in the context of zero-shot prompts. The role of literature expert does not hinder the bot from describing pizza to me, which makes me think the role parameter might be made obsolete. To test this theory, I prompted the bot to be an expert in pizza and then gave few shot examples of literature, and then asked it a question about literature. Because it's expertise is pizza and not literature, it did not learn to answer the question in the format provided. Instead it gives a description of the book long-form, like it did with pizza and the other zero-shot examples. This makes me think that when the role and shots are in conflict the shots are also disregarded. Interestingly, in this example the bot makes a contradiction about this tween drug propaganda book, saying that the book is a diary that is a work of fiction made up by the auther, and then saying it is based on the the story of a real girl. I experimented more with roles and shots and in the end tried to mix it up to see if the mixing the role as a pizza literature expert would make the chatbot respond creatively, however instead it narrowly replied "As this book does not focus on pizza or pizza toppings, there is no information regarding pizza toppings for "It Happened to Nancy." Creativity is a limit within the model we have created.

In [8]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in literature ."""}
]
print(return_OAIResponse("Who is the main character in The Great Gatsby?", context_user))

The main character in "The Great Gatsby" is Jay Gatsby. The novel, written by F. Scott Fitzgerald, is a story about the American Dream, wealth, love, and the Jazz Age in the 1920s. Jay Gatsby is a mysterious and enigmatic millionaire who throws lavish parties in the hope of rekindling his romance with Daisy Buchanan.


In [7]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in literature .

     Who is the main character in The Catcher in the Rye
    Author: J.D. Salinger
    Character: Holden Caulfield

     
     Who is the main character in The Bluest Eye
    Author: Toni Morrison
    Character: Pecola Breedlove"""}
]
print(return_OAIResponse("Who is the main character in The Great Gatsby?", context_user))

Author: F. Scott Fitzgerald
Character: Jay Gatsby


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in literature .

     Who is the main character in The Catcher in the Rye
    Author: J.D. Salinger
    Character: Holden Caulfield

     
     Who is the main character in The Bluest Eye
    Author: Toni Morrison
    Character: Pecola Breedlove"""}
]
print(return_OAIResponse("What is pizza?", context_user))

Pizza is a popular dish that typically consists of a round, flattened base of dough topped with tomato sauce, cheese, and various toppings such as meats, vegetables, and herbs. It is baked in an oven and is a widely enjoyed food around the world.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in pizza .

     Who is the main character in The Catcher in the Rye
    Author: J.D. Salinger
    Character: Holden Caulfield

     
     Who is the main character in The Bluest Eye
    Author: Toni Morrison
    Character: Pecola Breedlove"""}
]
print(return_OAIResponse("Who is the main character in It Happened to Nancy?", context_user))

"It Happened to Nancy" is a book presented as a real diary, but it is actually a work of fiction written by an anonymous author and based on the experiences of an actual teenage girl with HIV. The main character of the book is Nancy, a fictional teenage girl who contracts HIV through sexual assault.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in english literature .

    The Catcher in the rye is a book about what?
    Description: An angsty teenage boy on the verge of a nervous breakdown as he navigates the hypocrisy of his peer group and the adult world.
    Author: J.D. Salinger
    Character: Holden Caulfield

     
    The Bluest Eye is a book about what?
    Description: A book about the struggles of black women in the 1950s, a little black girl Pecola Breedlove is made to hate herself so thoroughly by her abusive family and society, that, after being raped and impregnated by her father,  has a nervous breakdown and hallucinates that her eyes are blue. 
    Author: Toni Morrison
    Character: Pecola Breedlove"""}
]
print(return_OAIResponse("It Happened to Nancy is a book about what?", context_user))

"It Happened to Nancy" is a 1994 book that is based on the diaries of a young girl who contracted HIV through a single sexual encounter. The book deals with her struggle to cope with the diagnosis, the stigma surrounding the virus, and her eventual death. It is a harrowing and emotional read that sheds light on the impact of HIV/AIDS on individuals and society.


In [14]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in english literature .

    Who is the main character in The Catcher in the Rye?
    Description: An angsty teenage boy on the verge of a nervous breakdown as he navigates the hypocrisy of his peer group and the adult world.
    Author: J.D. Salinger
    Character: Holden Caulfield

     
        Who is the main character in The Bluest Eye?
    Description: A book about the struggles of black women in the 1950s, a little black girl Pecola Breedlove is made to hate herself so thoroughly by her abusive family and society, that, after being raped and impregnated by her father,  has a nervous breakdown and hallucinates that her eyes are blue. 
    Author: Toni Morrison
    Character: Pecola Breedlove"""}
]
print(return_OAIResponse("is It Happened to Nancy a true story?", context_user))

It Happened to Nancy is a novel based on a true story. The book is written in the form of a diary and follows the story of a teenage girl named Nancy who contracts HIV/AIDS through unprotected sex. The novel was inspired by the real-life diary of a girl who died from AIDS.


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in pizza literature.

    Who is the main character in The Catcher in the Rye?
    Description: An angsty teenage boy on the verge of a nervous breakdown as he navigates the hypocrisy of his peer group and the adult world.
    Author: J.D. Salinger
    Character: Holden Caulfield

     
    Who is the main character in The Bluest Eye?
    Description: A book about the struggles of black women in the 1950s, a little black girl Pecola Breedlove is made to hate herself so thoroughly by her abusive family and society, that, after being raped and impregnated by her father,  has a nervous breakdown and hallucinates that her eyes are blue. 
    Author: Toni Morrison
    Character: Pecola Breedlove"""}
]
print(return_OAIResponse("what is are the pizza toppings for It Happened to Nancy?", context_user))

Book Title: It Happened to Nancy
Description: A novel in diary form that follows the story of a teenage girl who becomes infected with HIV after being raped at a party.
Author: Beatrice Sparks

As this book does not focus on pizza or pizza toppings, there is no information regarding pizza toppings for "It Happened to Nancy."
